In [54]:
import re, nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
import joblib
import pandas as pd
import json



df = pd.read_csv("C:/Users/ttt56/Desktop/Project-LorcanO'Mahony/merged_data.csv")


display(df.head())
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
display(df.head(100))   

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ttt56\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ttt56\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ttt56\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ttt56\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,name,url,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Euro_NCAP_Overall_Safety_Rating,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,Boot_Volume,Ground_Clearance,County,Fuel_Tank_Capacity,Engine_Power,Top_Speed,Battery_Capacity,Acceleration,section,combined,reddit_content,sentiment,sentiment3,sentiment2
0,Hyundai Tucson 2016 Executive,https://www.donedeal.ie/cars-for-sale/hyundai-...,Executive,"128,000 mi",1.7 L Diesel,Jun 2026,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.0 L/100 km,4.4 L/100 km,5.1 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,5/5,85%,71%,86%,71%,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,"€10,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Hyundai Tucson,"Hyundai Tucson Reliability? | Boys, I have a d...",0.160085,2,2
1,Audi Q5 40 TDI Quattro S Line 5dr S Tronic,https://www.donedeal.ie/cars-for-sale/audi-q5-...,---,"45,139 mi",2.0 L Diesel,---,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,"£27,740",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Audi Q5,Considering Audi Q5? | Just curious what your...,0.214252,1,3
2,2018 AUDI A6 DIESEL SALOON 2.0 TDI ULTRA S- LINE,https://www.donedeal.ie/cars-for-sale/2018-aud...,S Line,"81,300 mi",2.0 L Diesel,Jun 2026,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,3.9 L/100 km,4.4 L/100 km,5.2 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,5/5,83%,86%,91%,41%,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,"€22,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Audi A6,Will I regret this Audi A6 | I bought this 202...,0.416667,2,5
3,"Suzuki Swift 2017, Auto, Incredibly Low Mileage",https://www.donedeal.ie/cars-for-sale/suzuki-s...,SZ5,"27,654 km",1.2 L Petrol Hybrid,Oct 2025,2017.0,Automatic,3,Suzuki,Petrol Hybrid,Hatchback,5,Luxury Trim,5,Black,200.0,4.2 L/100 km,4.5 L/100 km,4.9 L/100 km,1.2 L,3.84 m,1.50 m,1.74 m,3/5,75%,25%,83%,69%,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,"€11,000",265 L,12 cm,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Suzuki Swift,Is the new Suzuki Swift unsafe? | 1 star ratin...,-0.153409,1,1
4,Peugeot 3008 1.5 Bluehdi 130bhp Auto 6.4 GT,https://www.donedeal.ie/cars-for-sale/peugeot-...,GT,"19,448 km",1.5 L Diesel,Jan 2028,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,NaN,5.3 L/100 km,NaN,1.5 L,4.45 m,NaN,1.84 m,5/5,85%,58%,86%,67%,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,"€40,995",NaN,NaN,Co. Dublin,NaN,NaN,NaN,NaN,NaN,selection1,Peugeot 3008,Is 3008 a good buy? Haven’t owned Peugeot befo...,0.333333,2,4


,name,url,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Euro_NCAP_Overall_Safety_Rating,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,Boot_Volume,Ground_Clearance,County,Fuel_Tank_Capacity,Engine_Power,Top_Speed,Battery_Capacity,Acceleration,section,combined,reddit_content,sentiment,sentiment3,sentiment2
0,Hyundai Tucson 2016 Executive,https://www.donedeal.ie/cars-for-sale/hyundai-...,Executive,"128,000 mi",1.7 L Diesel,Jun 2026,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.0 L/100 km,4.4 L/100 km,5.1 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,5/5,85%,71%,86%,71%,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,"€10,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Hyundai Tucson,"Hyundai Tucson Reliability? | Boys, I have a d...",0.160085,2,2
1,Audi Q5 40 TDI Quattro S Line 5dr S Tronic,https://www.donedeal.ie/cars-for-sale/audi-q5-...,---,"45,139 mi",2.0 L Diesel,---,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,"£27,740",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Audi Q5,Considering Audi Q5? | Just curious what your...,0.214252,1,3
2,2018 AUDI A6 DIESEL SALOON 2.0 TDI ULTRA S- LINE,https://www.donedeal.ie/cars-for-sale/2018-aud...,S Line,"81,300 mi",2.0 L Diesel,Jun 2026,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,3.9 L/100 km,4.4 L/100 km,5.2 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,5/5,83%,86%,91%,41%,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,"€22,500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Audi A6,Will I regret this Audi A6 | I bought this 202...,0.416667,2,5
3,"Suzuki Swift 2017, Auto, Incredibly Low Mileage",https://www.donedeal.ie/cars-for-sale/suzuki-s...,SZ5,"27,654 km",1.2 L Petrol Hybrid,Oct 2025,2017.0,Automatic,3,Suzuki,Petrol Hybrid,Hatchback,5,Luxury Trim,5,Black,200.0,4.2 L/100 km,4.5 L/100 km,4.9 L/100 km,1.2 L,3.84 m,1.50 m,1.74 m,3/5,75%,25%,83%,69%,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,"€11,000",265 L,12 cm,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Suzuki Swift,Is the new Suzuki Swift unsafe? | 1 star ratin...,-0.153409,1,1
4,Peugeot 3008 1.5 Bluehdi 130bhp Auto 6.4 GT,https://www.donedeal.ie/cars-for-sale/peugeot-...,GT,"19,448 km",1.5 L Diesel,Jan 2028,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,NaN,5.3 L/100 km,NaN,1.5 L,4.45 m,NaN,1.84 m,5/5,85%,58%,86%,67%,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,"€40,995",NaN,NaN,Co. Dublin,NaN,NaN,NaN,NaN,NaN,selection1,Peugeot 3008,Is 3008 a good buy? Haven’t owned Peugeot befo...,0.333333,2,4
5,Toyota Yaris 1.0 Luna 4DR,https://www.donedeal.ie/cars-for-sale/toyota-y...,Luna,"80,000 km",1.0 L Petrol,Jun 2026,2016.0,Manual,2,Toyota,Petrol,Hatchback,4,Medium Trim,5,Blue,180.0,3.8 L/100 km,4.3 L/100 km,5.2 L/100 km,1.0 L,3.95 m,1.51 m,1.70 m,5/5,81%,86%,89%,60%,Ireland,No,Franchise Dealer,"Macroom, Co. Cork",Yaris,"€13,000",768 L,14 cm,Co. Cork,42 L,NaN,NaN,NaN,NaN,selection1,Toyota Yaris,Why does everyone recommend the Toyota Yaris o...,0.350000,2,4
6,Volkswagen Polo Lovely Spec R-line Polo With Allo,https://www.donedeal.ie/cars-for-sale/volkswag...,R-Line,"20,800 km",1.0 L Petrol,Jul 2026,2022.0,Manual,2,Volkswagen,Petrol,Hatchback,5,Sports Trim,5,White,190.0,NaN,5.2 L/100 km,NaN,1.0 L,4.07 m,NaN,1.75 m,5/5,85%,59%,96%,76%,Ireland,No,Franchise Dealer\n4.6/5,Co. Galway,Polo,"€24,945","1,125 L",NaN,Co. Galway,40 L,NaN,NaN,NaN,NaN,selection1,Volkswagen Polo,Honest opinion on Volkswagen Polo? | Its a 2 p...,0.368182,2,1
7,Hyundai Santa Fe Premium AWD Phev 5DR Auto 7 Seat,https://www.doned

In [55]:
#PREPARING THE PRICE COLUMN BY REMOVING SPECIAL CHARACTERS AND CONVERTING THE POUNDS TO EUROS
import pandas as pd



numeric = pd.to_numeric(df['Price'].astype(str)
                        .str.replace(r'[^0-9.]', '', regex=True))
is_pound = df['Price'].str.contains('£', na=False)
df.loc[is_pound, 'Price'] = (numeric[is_pound] * 1.17).round(2)
df.loc[~is_pound, 'Price'] = numeric[~is_pound].round(2)

display(df.head(100))   




,name,url,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Euro_NCAP_Overall_Safety_Rating,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,Boot_Volume,Ground_Clearance,County,Fuel_Tank_Capacity,Engine_Power,Top_Speed,Battery_Capacity,Acceleration,section,combined,reddit_content,sentiment,sentiment3,sentiment2
0,Hyundai Tucson 2016 Executive,https://www.donedeal.ie/cars-for-sale/hyundai-...,Executive,"128,000 mi",1.7 L Diesel,Jun 2026,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.0 L/100 km,4.4 L/100 km,5.1 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,5/5,85%,71%,86%,71%,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Hyundai Tucson,"Hyundai Tucson Reliability? | Boys, I have a d...",0.160085,2,2
1,Audi Q5 40 TDI Quattro S Line 5dr S Tronic,https://www.donedeal.ie/cars-for-sale/audi-q5-...,---,"45,139 mi",2.0 L Diesel,---,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Audi Q5,Considering Audi Q5? | Just curious what your...,0.214252,1,3
2,2018 AUDI A6 DIESEL SALOON 2.0 TDI ULTRA S- LINE,https://www.donedeal.ie/cars-for-sale/2018-aud...,S Line,"81,300 mi",2.0 L Diesel,Jun 2026,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,3.9 L/100 km,4.4 L/100 km,5.2 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,5/5,83%,86%,91%,41%,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Audi A6,Will I regret this Audi A6 | I bought this 202...,0.416667,2,5
3,"Suzuki Swift 2017, Auto, Incredibly Low Mileage",https://www.donedeal.ie/cars-for-sale/suzuki-s...,SZ5,"27,654 km",1.2 L Petrol Hybrid,Oct 2025,2017.0,Automatic,3,Suzuki,Petrol Hybrid,Hatchback,5,Luxury Trim,5,Black,200.0,4.2 L/100 km,4.5 L/100 km,4.9 L/100 km,1.2 L,3.84 m,1.50 m,1.74 m,3/5,75%,25%,83%,69%,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,265 L,12 cm,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Suzuki Swift,Is the new Suzuki Swift unsafe? | 1 star ratin...,-0.153409,1,1
4,Peugeot 3008 1.5 Bluehdi 130bhp Auto 6.4 GT,https://www.donedeal.ie/cars-for-sale/peugeot-...,GT,"19,448 km",1.5 L Diesel,Jan 2028,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,NaN,5.3 L/100 km,NaN,1.5 L,4.45 m,NaN,1.84 m,5/5,85%,58%,86%,67%,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,NaN,NaN,Co. Dublin,NaN,NaN,NaN,NaN,NaN,selection1,Peugeot 3008,Is 3008 a good buy? Haven’t owned Peugeot befo...,0.333333,2,4
5,Toyota Yaris 1.0 Luna 4DR,https://www.donedeal.ie/cars-for-sale/toyota-y...,Luna,"80,000 km",1.0 L Petrol,Jun 2026,2016.0,Manual,2,Toyota,Petrol,Hatchback,4,Medium Trim,5,Blue,180.0,3.8 L/100 km,4.3 L/100 km,5.2 L/100 km,1.0 L,3.95 m,1.51 m,1.70 m,5/5,81%,86%,89%,60%,Ireland,No,Franchise Dealer,"Macroom, Co. Cork",Yaris,13000.0,768 L,14 cm,Co. Cork,42 L,NaN,NaN,NaN,NaN,selection1,Toyota Yaris,Why does everyone recommend the Toyota Yaris o...,0.350000,2,4
6,Volkswagen Polo Lovely Spec R-line Polo With Allo,https://www.donedeal.ie/cars-for-sale/volkswag...,R-Line,"20,800 km",1.0 L Petrol,Jul 2026,2022.0,Manual,2,Volkswagen,Petrol,Hatchback,5,Sports Trim,5,White,190.0,NaN,5.2 L/100 km,NaN,1.0 L,4.07 m,NaN,1.75 m,5/5,85%,59%,96%,76%,Ireland,No,Franchise Dealer\n4.6/5,Co. Galway,Polo,24945.0,"1,125 L",NaN,Co. Galway,40 L,NaN,NaN,NaN,NaN,selection1,Volkswagen Polo,Honest opinion on Volkswagen Polo? | Its a 2 p...,0.368182,2,1
7,Hyundai Santa Fe Premium AWD Phev 5DR Auto 7 Seat,https://www.donedeal.ie/cars-fo

In [ ]:

#DATA CLEANING


#I drop the following columns because they have too many null values 

columns_to_drop = ['url', 'name','reddit_content']
df = df.drop(columns_to_drop, axis=1)

print(df)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [57]:
df.head()

,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Euro_NCAP_Overall_Safety_Rating,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,Boot_Volume,Ground_Clearance,County,Fuel_Tank_Capacity,Engine_Power,Top_Speed,Battery_Capacity,Acceleration,section,combined,sentiment,sentiment3,sentiment2
0,Executive,"128,000 mi",1.7 L Diesel,Jun 2026,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.0 L/100 km,4.4 L/100 km,5.1 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,5/5,85%,71%,86%,71%,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Hyundai Tucson,0.160085,2,2
1,---,"45,139 mi",2.0 L Diesel,---,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Audi Q5,0.214252,1,3
2,S Line,"81,300 mi",2.0 L Diesel,Jun 2026,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,3.9 L/100 km,4.4 L/100 km,5.2 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,5/5,83%,86%,91%,41%,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Audi A6,0.416667,2,5
3,SZ5,"27,654 km",1.2 L Petrol Hybrid,Oct 2025,2017.0,Automatic,3,Suzuki,Petrol Hybrid,Hatchback,5,Luxury Trim,5,Black,200.0,4.2 L/100 km,4.5 L/100 km,4.9 L/100 km,1.2 L,3.84 m,1.50 m,1.74 m,3/5,75%,25%,83%,69%,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,265 L,12 cm,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Suzuki Swift,-0.153409,1,1
4,GT,"19,448 km",1.5 L Diesel,Jan 2028,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,NaN,5.3 L/100 km,NaN,1.5 L,4.45 m,NaN,1.84 m,5/5,85%,58%,86%,67%,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,NaN,NaN,Co. Dublin,NaN,NaN,NaN,NaN,NaN,selection1,Peugeot 3008,0.333333,2,4


In [58]:
import pandas as pd
import numpy as np
import re
import datetime as dt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor


In [59]:
#CONVERT MILES TO KM AND REMOVE LETTERS SO THE DATA IS ONLY NUMERIC

import pandas as pd
import numpy as np
import re

def to_km(mileage_str):
    if pd.isna(mileage_str):
        return np.nan
    cleaned = re.sub(r'[^0-9.]', '', str(mileage_str))  
    if cleaned == '':
        return np.nan
    num = float(cleaned)
    return num * 1.60934 if 'mi' in str(mileage_str).lower() else num

df['Mileage'] = df['Mileage'].apply(to_km)

df.to_csv('cleaned_file.csv', index=False)

df.head()



,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Euro_NCAP_Overall_Safety_Rating,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,Boot_Volume,Ground_Clearance,County,Fuel_Tank_Capacity,Engine_Power,Top_Speed,Battery_Capacity,Acceleration,section,combined,sentiment,sentiment3,sentiment2
0,Executive,205995.52000,1.7 L Diesel,Jun 2026,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.0 L/100 km,4.4 L/100 km,5.1 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,5/5,85%,71%,86%,71%,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Hyundai Tucson,0.160085,2,2
1,---,72643.99826,2.0 L Diesel,---,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Audi Q5,0.214252,1,3
2,S Line,130839.34200,2.0 L Diesel,Jun 2026,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,3.9 L/100 km,4.4 L/100 km,5.2 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,5/5,83%,86%,91%,41%,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Audi A6,0.416667,2,5
3,SZ5,27654.00000,1.2 L Petrol Hybrid,Oct 2025,2017.0,Automatic,3,Suzuki,Petrol Hybrid,Hatchback,5,Luxury Trim,5,Black,200.0,4.2 L/100 km,4.5 L/100 km,4.9 L/100 km,1.2 L,3.84 m,1.50 m,1.74 m,3/5,75%,25%,83%,69%,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,265 L,12 cm,NaN,NaN,NaN,NaN,NaN,NaN,selection1,Suzuki Swift,-0.153409,1,1
4,GT,19448.00000,1.5 L Diesel,Jan 2028,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,NaN,5.3 L/100 km,NaN,1.5 L,4.45 m,NaN,1.84 m,5/5,85%,58%,86%,67%,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,NaN,NaN,Co. Dublin,NaN,NaN,NaN,NaN,NaN,selection1,Peugeot 3008,0.333333,2,4


In [60]:

df = df.drop('section', axis=1)
df.head()

,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Rural_Fuel_Consumption,Comb_Fuel_Consumption,Urban_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Euro_NCAP_Overall_Safety_Rating,Child_Protection_Rating,Safety_Assistance_Rating,Adult_Protection_Rating,Pedestrian_Protection_Rating,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,Boot_Volume,Ground_Clearance,County,Fuel_Tank_Capacity,Engine_Power,Top_Speed,Battery_Capacity,Acceleration,combined,sentiment,sentiment3,sentiment2
0,Executive,205995.52000,1.7 L Diesel,Jun 2026,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.0 L/100 km,4.4 L/100 km,5.1 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,5/5,85%,71%,86%,71%,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hyundai Tucson,0.160085,2,2
1,---,72643.99826,2.0 L Diesel,---,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Audi Q5,0.214252,1,3
2,S Line,130839.34200,2.0 L Diesel,Jun 2026,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,3.9 L/100 km,4.4 L/100 km,5.2 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,5/5,83%,86%,91%,41%,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Audi A6,0.416667,2,5
3,SZ5,27654.00000,1.2 L Petrol Hybrid,Oct 2025,2017.0,Automatic,3,Suzuki,Petrol Hybrid,Hatchback,5,Luxury Trim,5,Black,200.0,4.2 L/100 km,4.5 L/100 km,4.9 L/100 km,1.2 L,3.84 m,1.50 m,1.74 m,3/5,75%,25%,83%,69%,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,265 L,12 cm,NaN,NaN,NaN,NaN,NaN,NaN,Suzuki Swift,-0.153409,1,1
4,GT,19448.00000,1.5 L Diesel,Jan 2028,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,NaN,5.3 L/100 km,NaN,1.5 L,4.45 m,NaN,1.84 m,5/5,85%,58%,86%,67%,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,NaN,NaN,Co. Dublin,NaN,NaN,NaN,NaN,NaN,Peugeot 3008,0.333333,2,4


In [61]:
#DROPPING ALL COLUMNS WITH UNDER 60% NAN VALUES 

hi_na_threshold = 0.60
hi_na_cols      = df.columns[df.isna().mean() > hi_na_threshold]

df.drop(columns = hi_na_cols, inplace=True)

df.head()


,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2
0,Executive,205995.52000,1.7 L Diesel,Jun 2026,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.4 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,Hyundai Tucson,0.160085,2,2
1,---,72643.99826,2.0 L Diesel,---,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,Audi Q5,0.214252,1,3
2,S Line,130839.34200,2.0 L Diesel,Jun 2026,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.4 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,Audi A6,0.416667,2,5
3,SZ5,27654.00000,1.2 L Petrol Hybrid,Oct 2025,2017.0,Automatic,3,Suzuki,Petrol Hybrid,Hatchback,5,Luxury Trim,5,Black,200.0,4.5 L/100 km,1.2 L,3.84 m,1.50 m,1.74 m,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,NaN,Suzuki Swift,-0.153409,1,1
4,GT,19448.00000,1.5 L Diesel,Jan 2028,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.3 L/100 km,1.5 L,4.45 m,NaN,1.84 m,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4


In [ ]:


#CHECK THE FUEL TYPE IN THE ENGINE SIZE COLUMN AND UPDATE THE FUEL TYPE COLUMN WITH THE FUEL TYPE OF THE ENGINE IF IT IS EMPTY

import numpy as np
import pandas as pd

df2 = df

eng_lower = df2["Engine"].astype(str).str.lower()

conditions = [
    eng_lower.str.contains("petrol"),
    eng_lower.str.contains("diesel"),
]

choices = ["Petrol", "Diesel"]

df2["Fuel_Type"] = np.select(conditions, choices, default=df["Fuel_Type"])

pd.set_option("display.max_columns", None)


pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
display(df2.head(100))        


,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2
0,Executive,205995.52000,1.7 L Diesel,Jun 2026,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.4 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,Hyundai Tucson,0.160085,2,2
1,---,72643.99826,2.0 L Diesel,---,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,Audi Q5,0.214252,1,3
2,S Line,130839.34200,2.0 L Diesel,Jun 2026,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.4 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,Audi A6,0.416667,2,5
3,SZ5,27654.00000,1.2 L Petrol Hybrid,Oct 2025,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.5 L/100 km,1.2 L,3.84 m,1.50 m,1.74 m,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,NaN,Suzuki Swift,-0.153409,1,1
4,GT,19448.00000,1.5 L Diesel,Jan 2028,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.3 L/100 km,1.5 L,4.45 m,NaN,1.84 m,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4
5,Luna,80000.00000,1.0 L Petrol,Jun 2026,2016.0,Manual,2,Toyota,Petrol,Hatchback,4,Medium Trim,5,Blue,180.0,4.3 L/100 km,1.0 L,3.95 m,1.51 m,1.70 m,Ireland,No,Franchise Dealer,"Macroom, Co. Cork",Yaris,13000.0,Co. Cork,Toyota Yaris,0.350000,2,4
6,R-Line,20800.00000,1.0 L Petrol,Jul 2026,2022.0,Manual,2,Volkswagen,Petrol,Hatchback,5,Sports Trim,5,White,190.0,5.2 L/100 km,1.0 L,4.07 m,NaN,1.75 m,Ireland,No,Franchise Dealer\n4.6/5,Co. Galway,Polo,24945.0,Co. Galway,Volkswagen Polo,0.368182,2,1
7,Premium,18461.00000,1.6 L Petrol Plug-in Hybrid,Aug 2027,2023.0,Automatic,1,Hyundai,Petrol,SUV,5,Medium Trim,7,Grey,NaN,NaN,1.6 L,4.79 m,1.71 m,1.90 m,Ireland,No,Franchise Dealer,Co. Kerry,Santa Fe,57950.0,Co. Kerry,Hyundai Santa Fe,0.341522,2,4
8,Ambition,320000.00000,1.6 L Diesel,Mar 2027,2016.0,Manual,3,Skoda,Diesel,Saloon,4,Medium Trim,5,Silver,180.0,4.1 L/100 km,1.6 L,4.66 m,1.46 m,1.81 m,Ireland,No,Private Seller\nNo rating yet,"North County, Co. Dublin",Octavia,6250.0,NaN,Skoda Octavia,0.116796,1,2
9,Luna,105000.00000,1.6 L Diesel,Mar 2027,2017.0,Manual,2,Toyota,Diesel,Saloon,4,Medium Trim,5,Grey,190.0,4.2 L/100 km,1.6 L,4.75 m,1.48 m,1.81 m,Ireland,No,Private Seller\nNo rating yet,"Ballina, Co. Mayo",Avensis,13100.0,NaN,Toyota Avensis,0.058966,1,2


In [63]:
#REMOVING EVERYTHING EXCEPT FOR NUMBERS FROM THE ENGINE COLUMN

df2['Engine'] = df['Engine'].str.extract(r'(\d*\.?\d+)')

df2.head()

,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2
0,Executive,205995.52000,1.7,Jun 2026,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.4 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,Hyundai Tucson,0.160085,2,2
1,---,72643.99826,2.0,---,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,Audi Q5,0.214252,1,3
2,S Line,130839.34200,2.0,Jun 2026,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.4 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,Audi A6,0.416667,2,5
3,SZ5,27654.00000,1.2,Oct 2025,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.5 L/100 km,1.2 L,3.84 m,1.50 m,1.74 m,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,NaN,Suzuki Swift,-0.153409,1,1
4,GT,19448.00000,1.5,Jan 2028,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.3 L/100 km,1.5 L,4.45 m,NaN,1.84 m,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4


In [64]:


#REMOVE ALL ROWS WITH NO PRICE IN IT
df2 = df2[
    pd.to_numeric(df['Price'], errors='coerce').notna()   
    & ~df2['Price'].isin([0, 1])                         
]




In [ ]:
df2.head(20)  

,Trim,Mileage,Engine,NCT_Expiry,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2
0,Executive,205995.52000,1.7,Jun 2026,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.4 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,Hyundai Tucson,0.160085,2,2
1,---,72643.99826,2.0,---,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,Audi Q5,0.214252,1,3
2,S Line,130839.34200,2.0,Jun 2026,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.4 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,Audi A6,0.416667,2,5
3,SZ5,27654.00000,1.2,Oct 2025,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.5 L/100 km,1.2 L,3.84 m,1.50 m,1.74 m,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,NaN,Suzuki Swift,-0.153409,1,1
4,GT,19448.00000,1.5,Jan 2028,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.3 L/100 km,1.5 L,4.45 m,NaN,1.84 m,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4
5,Luna,80000.00000,1.0,Jun 2026,2016.0,Manual,2,Toyota,Petrol,Hatchback,4,Medium Trim,5,Blue,180.0,4.3 L/100 km,1.0 L,3.95 m,1.51 m,1.70 m,Ireland,No,Franchise Dealer,"Macroom, Co. Cork",Yaris,13000.0,Co. Cork,Toyota Yaris,0.350000,2,4
6,R-Line,20800.00000,1.0,Jul 2026,2022.0,Manual,2,Volkswagen,Petrol,Hatchback,5,Sports Trim,5,White,190.0,5.2 L/100 km,1.0 L,4.07 m,NaN,1.75 m,Ireland,No,Franchise Dealer\n4.6/5,Co. Galway,Polo,24945.0,Co. Galway,Volkswagen Polo,0.368182,2,1
7,Premium,18461.00000,1.6,Aug 2027,2023.0,Automatic,1,Hyundai,Petrol,SUV,5,Medium Trim,7,Grey,NaN,NaN,1.6 L,4.79 m,1.71 m,1.90 m,Ireland,No,Franchise Dealer,Co. Kerry,Santa Fe,57950.0,Co. Kerry,Hyundai Santa Fe,0.341522,2,4
8,Ambition,320000.00000,1.6,Mar 2027,2016.0,Manual,3,Skoda,Diesel,Saloon,4,Medium Trim,5,Silver,180.0,4.1 L/100 km,1.6 L,4.66 m,1.46 m,1.81 m,Ireland,No,Private Seller\nNo rating yet,"North County, Co. Dublin",Octavia,6250.0,NaN,Skoda Octavia,0.116796,1,2
9,Luna,105000.00000,1.6,Mar 2027,2017.0,Manual,2,Toyota,Diesel,Saloon,4,Medium Trim,5,Grey,190.0,4.2 L/100 km,1.6 L,4.75 m,1.48 m,1.81 m,Ireland,No,Private Seller\nNo rating yet,"Ballina, Co. Mayo",Avensis,13100.0,NaN,Toyota Avensis,0.058966,1,2


In [66]:
#CONVERTING NCT EXPIRY DATE INTO DATE FORMAT AND THEN CONVERTING IT INTO DAYS UNTIL NCT EXPIRY


import pandas as pd
from datetime import datetime

today = pd.Timestamp('today').normalize()

def parse_nct(val: str):
    val = str(val).strip()
    if val in {"---", "", "nan"}:
        return pd.NaT
    try:
        return pd.to_datetime(val, format="%b %Y") + pd.offsets.MonthEnd(0)
    except ValueError:
        return pd.NaT

df2["NCT_Expiry_Date"] = df2["NCT_Expiry"].apply(parse_nct)

df2["Days_Until_NCT"] = (df2["NCT_Expiry_Date"] - today).dt.days

df2["NCT_Expired"] = df2["Days_Until_NCT"] < 0


df2 = df2.drop(columns=["NCT_Expiry"])

df2.head()


C:\Users\ttt56\AppData\Local\Temp\ipykernel_20672\2800788229.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["NCT_Expiry_Date"] = df2["NCT_Expiry"].apply(parse_nct)
C:\Users\ttt56\AppData\Local\Temp\ipykernel_20672\2800788229.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Days_Until_NCT"] = (df2["NCT_Expiry_Date"] - today).dt.days
C:\Users\ttt56\AppData\Local\Temp\ipykernel_20672\2800788229.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,NCT_Expiry_Date,Days_Until_NCT,NCT_Expired
0,Executive,205995.52000,1.7,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.4 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,Hyundai Tucson,0.160085,2,2,2026-06-30,416.0,False
1,---,72643.99826,2.0,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,Audi Q5,0.214252,1,3,NaT,NaN,False
2,S Line,130839.34200,2.0,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.4 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,Audi A6,0.416667,2,5,2026-06-30,416.0,False
3,SZ5,27654.00000,1.2,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.5 L/100 km,1.2 L,3.84 m,1.50 m,1.74 m,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,NaN,Suzuki Swift,-0.153409,1,1,2025-10-31,174.0,False
4,GT,19448.00000,1.5,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.3 L/100 km,1.5 L,4.45 m,NaN,1.84 m,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4,2028-01-31,996.0,False


In [67]:
df2 = df2.drop(columns=["NCT_Expiry_Date"])


df2.head(100)


,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired
0,Executive,205995.52000,1.7,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.4 L/100 km,1.7 L,4.48 m,1.65 m,1.85 m,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,Hyundai Tucson,0.160085,2,2,416.0,False
1,---,72643.99826,2.0,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,Audi Q5,0.214252,1,3,NaN,False
2,S Line,130839.34200,2.0,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.4 L/100 km,2.0 L,4.93 m,1.46 m,1.87 m,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,Audi A6,0.416667,2,5,416.0,False
3,SZ5,27654.00000,1.2,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.5 L/100 km,1.2 L,3.84 m,1.50 m,1.74 m,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,NaN,Suzuki Swift,-0.153409,1,1,174.0,False
4,GT,19448.00000,1.5,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.3 L/100 km,1.5 L,4.45 m,NaN,1.84 m,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False
5,Luna,80000.00000,1.0,2016.0,Manual,2,Toyota,Petrol,Hatchback,4,Medium Trim,5,Blue,180.0,4.3 L/100 km,1.0 L,3.95 m,1.51 m,1.70 m,Ireland,No,Franchise Dealer,"Macroom, Co. Cork",Yaris,13000.0,Co. Cork,Toyota Yaris,0.350000,2,4,416.0,False
6,R-Line,20800.00000,1.0,2022.0,Manual,2,Volkswagen,Petrol,Hatchback,5,Sports Trim,5,White,190.0,5.2 L/100 km,1.0 L,4.07 m,NaN,1.75 m,Ireland,No,Franchise Dealer\n4.6/5,Co. Galway,Polo,24945.0,Co. Galway,Volkswagen Polo,0.368182,2,1,447.0,False
7,Premium,18461.00000,1.6,2023.0,Automatic,1,Hyundai,Petrol,SUV,5,Medium Trim,7,Grey,NaN,NaN,1.6 L,4.79 m,1.71 m,1.90 m,Ireland,No,Franchise Dealer,Co. Kerry,Santa Fe,57950.0,Co. Kerry,Hyundai Santa Fe,0.341522,2,4,843.0,False
8,Ambition,320000.00000,1.6,2016.0,Manual,3,Skoda,Diesel,Saloon,4,Medium Trim,5,Silver,180.0,4.1 L/100 km,1.6 L,4.66 m,1.46 m,1.81 m,Ireland,No,Private Seller\nNo rating yet,"North County, Co. Dublin",Octavia,6250.0,NaN,Skoda Octavia,0.116796,1,2,690.0,False
9,Luna,105000.00000,1.6,2017.0,Manual,2,Toyota,Diesel,Saloon,4,Medium Trim,5,Grey,190.0,4.2 L/100 km,1.6 L,4.75 m,1.48 m,1.81 m,Ireland,No,Private Seller\nNo rating yet,"Ballina, Co. Mayo",Avensis,13100.0,NaN,Toyota Avensis,0.058966,1,2,690.0,False


In [68]:
#REMOVING OUTLIERS 

df2 = df2[(df2['Price'] >= 100) & (df2['Price'] <= 300000)]


In [14]:


df2.to_excel('df2.xlsx', index=False, engine='openpyxl')

In [69]:
from pandas import  read_csv, DataFrame
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from pandas import read_csv, DataFrame
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [70]:
df2.head(7)
df2.shape
print(df2.info())
print(df2.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 12537 entries, 0 to 13053
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Trim                    12537 non-null  object 
 1   Mileage                 12249 non-null  float64
 2   Engine                  12402 non-null  object 
 3   Year                    12518 non-null  float64
 4   Transmission            12518 non-null  object 
 5   Total_Owners            12536 non-null  object 
 6   Make                    12537 non-null  object 
 7   Fuel_Type               12528 non-null  object 
 8   Body_Type               11401 non-null  object 
 9   Doors                   11401 non-null  object 
 10  Trim_Level              11401 non-null  object 
 11  Seats                   11401 non-null  object 
 12  Colour                  11401 non-null  object 
 13  Road_Tax                8851 non-null   float64
 14  Comb_Fuel_Consumption   6304 non-null   obj

In [71]:
df2.head(7)
df2.shape
print(df2.info())
print(df2.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 12537 entries, 0 to 13053
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Trim                    12537 non-null  object 
 1   Mileage                 12249 non-null  float64
 2   Engine                  12402 non-null  object 
 3   Year                    12518 non-null  float64
 4   Transmission            12518 non-null  object 
 5   Total_Owners            12536 non-null  object 
 6   Make                    12537 non-null  object 
 7   Fuel_Type               12528 non-null  object 
 8   Body_Type               11401 non-null  object 
 9   Doors                   11401 non-null  object 
 10  Trim_Level              11401 non-null  object 
 11  Seats                   11401 non-null  object 
 12  Colour                  11401 non-null  object 
 13  Road_Tax                8851 non-null   float64
 14  Comb_Fuel_Consumption   6304 non-null   obj

In [72]:
#HAVE TO DOUBLE CHECK THAT ALL VALUES IN PRICE ARE NUMERIC

df_raw = pd.read_excel('df2.xlsx')

print('dtype straight from disk:', df_raw['Price'].dtype)
suspicious = df_raw['Price'].astype(str).loc[
    df_raw['Price'].astype(str).str.contains(r'[^0-9.\s,]', regex=True)
]
print('non-numeric raw values:\n', suspicious.unique()[:20])



FileNotFoundError: [Errno 2] No such file or directory: 'df2.xlsx'

In [73]:
#HAVE TO DOUBLE CHECK THAT ALL VALUES IN PRICE ARE NUMERIC


df2['Price'].dtype 
df2['Price'].apply(type).value_counts()  

Price
<class 'float'>    12537
Name: count, dtype: int64

In [74]:
#HAVE TO DOUBLE CHECK THAT ALL VALUES IN PRICE ARE NUMERIC

df2['Price'] = pd.to_numeric(df['Price'], errors='coerce')


In [75]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [76]:
#REMOVE THE M FROM External_Length	External_Height	External_Width COLUMNS AND THEY WILL BE NUMERICAL


cols_to_clean = [
    'External_Length',
    'External_Height',
    'External_Width',
    'Comb_Fuel_Consumption'  
]

for col in cols_to_clean:
    df2[col] = (
        df2[col]
          .astype(str)
          .str.replace(r'[^0-9.]', '', regex=True) 
          .replace('', np.nan)    
          .astype(float)
    )




In [77]:
df2.head(20)


,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,Engine_Size,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired
0,Executive,205995.52000,1.7,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,1.7 L,4.48,1.65,1.85,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.00,NaN,Hyundai Tucson,0.160085,2,2,416.0,False
1,---,72643.99826,2.0,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.80,NaN,Audi Q5,0.214252,1,3,NaN,False
2,S Line,130839.34200,2.0,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,2.0 L,4.93,1.46,1.87,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.00,NaN,Audi A6,0.416667,2,5,416.0,False
3,SZ5,27654.00000,1.2,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,1.2 L,3.84,1.50,1.74,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.00,NaN,Suzuki Swift,-0.153409,1,1,174.0,False
4,GT,19448.00000,1.5,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,1.5 L,4.45,NaN,1.84,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.00,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False
5,Luna,80000.00000,1.0,2016.0,Manual,2,Toyota,Petrol,Hatchback,4,Medium Trim,5,Blue,180.0,4.31,1.0 L,3.95,1.51,1.70,Ireland,No,Franchise Dealer,"Macroom, Co. Cork",Yaris,13000.00,Co. Cork,Toyota Yaris,0.350000,2,4,416.0,False
6,R-Line,20800.00000,1.0,2022.0,Manual,2,Volkswagen,Petrol,Hatchback,5,Sports Trim,5,White,190.0,5.21,1.0 L,4.07,NaN,1.75,Ireland,No,Franchise Dealer\n4.6/5,Co. Galway,Polo,24945.00,Co. Galway,Volkswagen Polo,0.368182,2,1,447.0,False
7,Premium,18461.00000,1.6,2023.0,Automatic,1,Hyundai,Petrol,SUV,5,Medium Trim,7,Grey,NaN,NaN,1.6 L,4.79,1.71,1.90,Ireland,No,Franchise Dealer,Co. Kerry,Santa Fe,57950.00,Co. Kerry,Hyundai Santa Fe,0.341522,2,4,843.0,False
8,Ambition,320000.00000,1.6,2016.0,Manual,3,Skoda,Diesel,Saloon,4,Medium Trim,5,Silver,180.0,4.11,1.6 L,4.66,1.46,1.81,Ireland,No,Private Seller\nNo rating yet,"North County, Co. Dublin",Octavia,6250.00,NaN,Skoda Octavia,0.116796,1,2,690.0,False
9,Luna,105000.00000,1.6,2017.0,Manual,2,Toyota,Diesel,Saloon,4,Medium Trim,5,Grey,190.0,4.21,1.6 L,4.75,1.48,1.81,Ireland,No,Private Seller\nNo rating yet,"Ballina, Co. Mayo",Avensis,13100.00,NaN,Toyota Avensis,0.058966,1,2,690.0,False


In [78]:

import numpy as np
import pandas as pd

RAW_COL   = 'Engine_Size'
TARGET    = 'Engine'

df2['Engine_Size_clean'] = (
    df2[RAW_COL]                
        .astype(str)
        .str.replace(r'[^0-9.]', '', regex=True)
        .replace({'': np.nan, 'nan': np.nan})      
        .astype(float)
)

df2[TARGET] = df2[TARGET].fillna(df2['Engine_Size_clean'])

df2 = df2.drop(columns=[RAW_COL, 'Engine_Size_clean'])


In [79]:
df2.head(20)


,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired
0,Executive,205995.52000,1.7,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.00,NaN,Hyundai Tucson,0.160085,2,2,416.0,False
1,---,72643.99826,2.0,2021.0,Automatic,---,Audi,Diesel,---,---,---,5,White,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.80,NaN,Audi Q5,0.214252,1,3,NaN,False
2,S Line,130839.34200,2.0,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.00,NaN,Audi A6,0.416667,2,5,416.0,False
3,SZ5,27654.00000,1.2,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.00,NaN,Suzuki Swift,-0.153409,1,1,174.0,False
4,GT,19448.00000,1.5,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,NaN,1.84,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.00,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False
5,Luna,80000.00000,1.0,2016.0,Manual,2,Toyota,Petrol,Hatchback,4,Medium Trim,5,Blue,180.0,4.31,3.95,1.51,1.70,Ireland,No,Franchise Dealer,"Macroom, Co. Cork",Yaris,13000.00,Co. Cork,Toyota Yaris,0.350000,2,4,416.0,False
6,R-Line,20800.00000,1.0,2022.0,Manual,2,Volkswagen,Petrol,Hatchback,5,Sports Trim,5,White,190.0,5.21,4.07,NaN,1.75,Ireland,No,Franchise Dealer\n4.6/5,Co. Galway,Polo,24945.00,Co. Galway,Volkswagen Polo,0.368182,2,1,447.0,False
7,Premium,18461.00000,1.6,2023.0,Automatic,1,Hyundai,Petrol,SUV,5,Medium Trim,7,Grey,NaN,NaN,4.79,1.71,1.90,Ireland,No,Franchise Dealer,Co. Kerry,Santa Fe,57950.00,Co. Kerry,Hyundai Santa Fe,0.341522,2,4,843.0,False
8,Ambition,320000.00000,1.6,2016.0,Manual,3,Skoda,Diesel,Saloon,4,Medium Trim,5,Silver,180.0,4.11,4.66,1.46,1.81,Ireland,No,Private Seller\nNo rating yet,"North County, Co. Dublin",Octavia,6250.00,NaN,Skoda Octavia,0.116796,1,2,690.0,False
9,Luna,105000.00000,1.6,2017.0,Manual,2,Toyota,Diesel,Saloon,4,Medium Trim,5,Grey,190.0,4.21,4.75,1.48,1.81,Ireland,No,Private Seller\nNo rating yet,"Ballina, Co. Mayo",Avensis,13100.00,NaN,Toyota Avensis,0.058966,1,2,690.0,False


In [80]:

unique = df2.nunique()

print(unique)



Trim                       497
Mileage                   6663
Engine                     106
Year                        39
Transmission                 5
Total_Owners                24
Make                        27
Fuel_Type                    7
Body_Type                   23
Doors                       10
Trim_Level                   5
Seats                       10
Colour                      28
Road_Tax                    58
Comb_Fuel_Consumption       82
External_Length            150
External_Height             60
External_Width              45
Current_Country_Of_Reg       4
Imported                     5
Independant_Dealer         280
Counties                   835
Model                      343
Price                     1997
County                      30
combined                   344
sentiment                  295
sentiment3                   2
sentiment2                   5
Days_Until_NCT             114
NCT_Expired                  2
dtype: int64


In [81]:
df2.describe()

,Mileage,Year,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Price,sentiment,sentiment3,sentiment2,Days_Until_NCT
count,1.224900e+04,12518.000000,8851.000000,6304.000000,8286.000000,6495.000000,8283.000000,12537.000000,12537.000000,12537.000000,12537.000000,10325.000000
mean,1.200867e+05,2017.439367,266.839453,4.851069,4.420168,1.543204,1.814064,20763.283794,0.150120,1.286991,2.855548,351.432058
std,9.050857e+04,4.992168,228.918855,6.773376,0.313467,0.106489,0.069836,17805.114268,0.190581,0.452375,1.377797,384.165549
min,0.000000e+00,1960.000000,0.000000,0.010000,3.430000,1.230000,1.600000,100.000000,-1.000000,1.000000,1.000000,-9141.000000
25%,4.900000e+04,2014.000000,190.000000,4.210000,4.230000,1.460000,1.780000,8250.000000,0.056395,1.000000,2.000000,143.000000
50%,1.026820e+05,2018.000000,200.000000,4.810000,4.410000,1.490000,1.810000,16750.000000,0.162785,1.000000,3.000000,294.000000
75%,1.800000e+05,2022.000000,270.000000,5.310000,4.670000,1.640000,1.850000,27950.000000,0.258247,2.000000,4.000000,631.000000
max,1.169316e+06,2025.000000,2400.000000,535.010000,5.200000,1.970000,2.210000,216976.500000,1.000000,2.000000,5.000000,1727.000000


In [27]:

"""
#DIVIDING THE COLUMNS UP BASED ON THE CORRECT ENCODING METHOD FOR EACH ONE


#MAPPING
 4   Transmission            12552 non-null  object - done
 7   Fuel_Type               12562 non-null  object - done
 21  Independant_Dealer      12552 non-null  object 
 31  NCT_Expired             12571 non-null  bool  - done 



#ONE HOT ENCODING
 10  Trim_Level              11434 non-null  object -
 19  Current_Country_Of_Reg  11530 non-null  object 
 20  Imported                10592 non-null  object - 



#CATEGORY ENCODING
 0   Trim                    12571 non-null  object 
 2   Engine                  12436 non-null  object 
 5   Total_Owners            12570 non-null  object 
 6   Make                    12571 non-null  object 
 8   Body_Type               11434 non-null  object 
 12  Colour                  11434 non-null  object 
 14  Comb_Fuel_Consumption   6322 non-null   object 
 15  Engine_Size             8425 non-null   object
 16  External_Length         8304 non-null   object 
 17  External_Height         6513 non-null   object
 18  External_Width          8301 non-null   object
 22  Counties                12571 non-null  object 
 23  Model                   12571 non-null  object
 25  County                  6673 non-null   object 
 26  combined                12571 non-null  object   

"""


'\n#DIVIDING THE COLUMNS UP BASED ON THE CORRECT ENCODING METHOD FOR EACH ONE\n\n\n#MAPPING\n 4   Transmission            12552 non-null  object - done\n 7   Fuel_Type               12562 non-null  object - done\n 21  Independant_Dealer      12552 non-null  object \n 31  NCT_Expired             12571 non-null  bool  - done \n\n\n\n#ONE HOT ENCODING\n 10  Trim_Level              11434 non-null  object -\n 19  Current_Country_Of_Reg  11530 non-null  object \n 20  Imported                10592 non-null  object - \n\n\n\n\n\n#CATEGORY ENCODING\n 0   Trim                    12571 non-null  object \n 2   Engine                  12436 non-null  object \n 5   Total_Owners            12570 non-null  object \n 6   Make                    12571 non-null  object \n 8   Body_Type               11434 non-null  object \n 12  Colour                  11434 non-null  object \n 14  Comb_Fuel_Consumption   6322 non-null   object \n 15  Engine_Size             8425 non-null   object\n 16  External_Length  

In [82]:
#REPLACE --- VALUES WITH NAN
df2.replace('---', np.nan, inplace=True)
df2.replace('-', np.nan, inplace=True)

df2.head(100)


,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired
0,Executive,205995.52000,1.7,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.00,NaN,Hyundai Tucson,0.160085,2,2,416.0,False
1,NaN,72643.99826,2.0,2021.0,Automatic,NaN,Audi,Diesel,NaN,NaN,NaN,5,White,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.80,NaN,Audi Q5,0.214252,1,3,NaN,False
2,S Line,130839.34200,2.0,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.00,NaN,Audi A6,0.416667,2,5,416.0,False
3,SZ5,27654.00000,1.2,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.00,NaN,Suzuki Swift,-0.153409,1,1,174.0,False
4,GT,19448.00000,1.5,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,NaN,1.84,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.00,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False
5,Luna,80000.00000,1.0,2016.0,Manual,2,Toyota,Petrol,Hatchback,4,Medium Trim,5,Blue,180.0,4.31,3.95,1.51,1.70,Ireland,No,Franchise Dealer,"Macroom, Co. Cork",Yaris,13000.00,Co. Cork,Toyota Yaris,0.350000,2,4,416.0,False
6,R-Line,20800.00000,1.0,2022.0,Manual,2,Volkswagen,Petrol,Hatchback,5,Sports Trim,5,White,190.0,5.21,4.07,NaN,1.75,Ireland,No,Franchise Dealer\n4.6/5,Co. Galway,Polo,24945.00,Co. Galway,Volkswagen Polo,0.368182,2,1,447.0,False
7,Premium,18461.00000,1.6,2023.0,Automatic,1,Hyundai,Petrol,SUV,5,Medium Trim,7,Grey,NaN,NaN,4.79,1.71,1.90,Ireland,No,Franchise Dealer,Co. Kerry,Santa Fe,57950.00,Co. Kerry,Hyundai Santa Fe,0.341522,2,4,843.0,False
8,Ambition,320000.00000,1.6,2016.0,Manual,3,Skoda,Diesel,Saloon,4,Medium Trim,5,Silver,180.0,4.11,4.66,1.46,1.81,Ireland,No,Private Seller\nNo rating yet,"North County, Co. Dublin",Octavia,6250.00,NaN,Skoda Octavia,0.116796,1,2,690.0,False
9,Luna,105000.00000,1.6,2017.0,Manual,2,Toyota,Diesel,Saloon,4,Medium Trim,5,Grey,190.0,4.21,4.75,1.48,1.81,Ireland,No,Private Seller\nNo rating yet,"Ballina, Co. Mayo",Avensis,13100.00,NaN,Toyota Avensis,0.058966,1,2,690.0,False


In [83]:
#DIVIDING THE INDEPENDANT DEALER COLUMN INTO THE TWO COLUMNS 'SELLER TYPE' AND  'SELLER RATING'

import pandas as pd
import numpy as np
import re


import pandas as pd


df2[['Seller_Type', 'Seller_Rating']] = df2['Independant_Dealer'].str.split(r'[\\\n]', n=1, expand=True)

df2['Seller_Rating'] = df2['Seller_Rating'].str.extract(r'(\d+\.\d+|\d+)').astype(float)

df2['Seller_Type'] = df2['Seller_Type'].replace('Independant Dealer', 'Independent Dealer')



df2.head(100)



,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Type,Seller_Rating
0,Executive,205995.52000,1.7,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.00,NaN,Hyundai Tucson,0.160085,2,2,416.0,False,Private Seller,NaN
1,NaN,72643.99826,2.0,2021.0,Automatic,NaN,Audi,Diesel,NaN,NaN,NaN,5,White,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.80,NaN,Audi Q5,0.214252,1,3,NaN,False,Franchise Dealer,NaN
2,S Line,130839.34200,2.0,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.00,NaN,Audi A6,0.416667,2,5,416.0,False,Verified Trader,NaN
3,SZ5,27654.00000,1.2,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.00,NaN,Suzuki Swift,-0.153409,1,1,174.0,False,Private Seller,NaN
4,GT,19448.00000,1.5,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,NaN,1.84,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.00,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False,Franchise Dealer,4.5
5,Luna,80000.00000,1.0,2016.0,Manual,2,Toyota,Petrol,Hatchback,4,Medium Trim,5,Blue,180.0,4.31,3.95,1.51,1.70,Ireland,No,Franchise Dealer,"Macroom, Co. Cork",Yaris,13000.00,Co. Cork,Toyota Yaris,0.350000,2,4,416.0,False,Franchise Dealer,NaN
6,R-Line,20800.00000,1.0,2022.0,Manual,2,Volkswagen,Petrol,Hatchback,5,Sports Trim,5,White,190.0,5.21,4.07,NaN,1.75,Ireland,No,Franchise Dealer\n4.6/5,Co. Galway,Polo,24945.00,Co. Galway,Volkswagen Polo,0.368182,2,1,447.0,False,Franchise Dealer,4.6
7,Premium,18461.00000,1.6,2023.0,Automatic,1,Hyundai,Petrol,SUV,5,Medium Trim,7,Grey,NaN,NaN,4.79,1.71,1.90,Ireland,No,Franchise Dealer,Co. Kerry,Santa Fe,57950.00,Co. Kerry,Hyundai Santa Fe,0.341522,2,4,843.0,False,Franchise Dealer,NaN
8,Ambition,320000.00000,1.6,2016.0,Manual,3,Skoda,Diesel,Saloon,4,Medium Trim,5,Silver,180.0,4.11,4.66,1.46,1.81,Ireland,No,Private Seller\nNo rating yet,"North County, Co. Dublin",Octavia,6250.00,NaN,Skoda Octavia,0.116796,1,2,690.0,False,Private Seller,NaN
9,Luna,105000.00000,1.6,2017.0,Manual,2,Toyota,Diesel,Saloon,4,Medium Trim,5,Grey,190.0,4.21,4.75,1.48,1.81,Ireland,No,Private Seller\nNo rating yet,"Ballina, Co. Mayo",Avensis,13100.00,NaN,Toyota Avensis,0.058966,1,2,690.0,False,Private Seller,NaN


In [84]:
df2.head()


,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Type,Seller_Rating
0,Executive,205995.52000,1.7,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,Hyundai Tucson,0.160085,2,2,416.0,False,Private Seller,NaN
1,NaN,72643.99826,2.0,2021.0,Automatic,NaN,Audi,Diesel,NaN,NaN,NaN,5,White,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,Audi Q5,0.214252,1,3,NaN,False,Franchise Dealer,NaN
2,S Line,130839.34200,2.0,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,Audi A6,0.416667,2,5,416.0,False,Verified Trader,NaN
3,SZ5,27654.00000,1.2,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,NaN,Suzuki Swift,-0.153409,1,1,174.0,False,Private Seller,NaN
4,GT,19448.00000,1.5,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,NaN,1.84,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False,Franchise Dealer,4.5


In [87]:

#REMOVING THE NUMBERS BETWEEN BRACKETS IN THE SELLER RATING CATEGORY

df2['Seller_Rating'] = df2['Seller_Rating'].str.replace(r"\(.*?\)", "", regex=True).str.strip()

df2.head(50)



AttributeError: Can only use .str accessor with string values!

In [88]:
df2.head()


,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Type,Seller_Rating
0,Executive,205995.52000,1.7,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,Hyundai Tucson,0.160085,2,2,416.0,False,Private Seller,NaN
1,NaN,72643.99826,2.0,2021.0,Automatic,NaN,Audi,Diesel,NaN,NaN,NaN,5,White,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,Audi Q5,0.214252,1,3,NaN,False,Franchise Dealer,NaN
2,S Line,130839.34200,2.0,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,Audi A6,0.416667,2,5,416.0,False,Verified Trader,NaN
3,SZ5,27654.00000,1.2,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,NaN,Suzuki Swift,-0.153409,1,1,174.0,False,Private Seller,NaN
4,GT,19448.00000,1.5,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,NaN,1.84,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False,Franchise Dealer,4.5


In [89]:
from pandas import read_csv, Series,DataFrame
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [127]:
df2.to_excel("df5_cleaned.xlsx", index=False)


In [90]:
#SHOWING ALL UNIQUE VALUES IN EACH COLUMN

for col in df2.columns:
    print(f"{col}:")
    print(df2[col].unique())
    print()   

Trim:
['Executive' nan 'S Line' 'SZ5' 'GT' 'Luna' 'R-Line' 'Premium' 'Ambition'
 'N' 'SV' 'S LINE' 'Trendline' 'Highline' 'SE' 'Edge' 'Strata' 'Terra'
 'Active' 'Platinum' 'Edition-R' 'Dynamic' 'BlueMotion' 'GLX'
 'COMFORTLINE' 'Classic' 'SE SELECT' 'STYLE' 'Elegance' 'S' 'xLine'
 'TRENDLINE' 'SC' 'Ambiente' 'IRISH EDITION' 'Comfortline' 'ALTERNATIVE'
 'BlueGT' 'GTE' 'SPORT' 'DYNAMIQUE NAV' 'AMBITION' 'TX' 'Style' 'Life'
 'Progressive' 'GTI' 'all' 'R-DESIGN' 'EX' 'DYNAMIQUE' 'SX'
 'Dynamique S Nav' 'SV Premium +SS' 'Sol' 'Luxury' 'Allure' 'Business'
 'Exclusive' 'XE' 'STEPWAY Comfort' 'Iconic' 'Sportline' 'PE'
 'Deluxe Plus' 'Prestige' 'Vogue' 'SV Premium' 'Titanium' 'Comfort'
 'M Sport' 'EXPRESSION+' 'Xperience' 'XCELLENCE' 'FEEL' 'K2' 'LX'
 'R-DYNAMIC S' 'VTR+' 'Zetec' 'REFERENCE' 'Deluxe' 'SIGNATURE' 'Enigma'
 'K3' 'Executive Plus' 'SE+' 'TREND & FUN' 'Extreme SE'
 'SPORT &amp; STYLE' 'Lounge' 'R-Dynamic S' 'Competition' 'K1' 'GT LINE'
 'L' 'Elite' 'Sport' 'R-Design' 'BLUEMOTION' 'L

In [214]:
df2.head(100)

,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Type,Seller_Rating
0,Executive,205995.52000,1.7,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,"Ardara, Co. Donegal",Tucson,10500.0,NaN,Hyundai Tucson,0.160085,2,2,419.0,False,Private Seller,NaN
1,NaN,72643.99826,2.0,2021.0,Automatic,NaN,Audi,Diesel,NaN,NaN,NaN,5,White,NaN,NaN,NaN,NaN,NaN,UK,NaN,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,Audi Q5,0.214252,1,3,NaN,False,Franchise Dealer,NaN
2,S Line,130839.34200,2.0,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,"Derry City, Co. Derry",A6,22500.0,NaN,Audi A6,0.416667,2,5,419.0,False,Verified Trader,NaN
3,SZ5,27654.00000,1.2,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,"Greystones, Co. Wicklow",Swift,11000.0,NaN,Suzuki Swift,-0.153409,1,1,177.0,False,Private Seller,NaN
4,GT,19448.00000,1.5,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,NaN,1.84,Ireland,No,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4,999.0,False,Franchise Dealer,4.5


In [91]:
#FIXING THE SELLER RATING COLUMN, ALL RATING ARE OUT OF 5 SO THIS DATA CAN BE REMOVED
df2['Seller_Rating_Clean'] = df2['Seller_Rating'].astype(str).str.split('/').str[0]

df2.head()

,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Type,Seller_Rating,Seller_Rating_Clean
0,Executive,205995.52000,1.7,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,Hyundai Tucson,0.160085,2,2,416.0,False,Private Seller,NaN,nan
1,NaN,72643.99826,2.0,2021.0,Automatic,NaN,Audi,Diesel,NaN,NaN,NaN,5,White,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,Audi Q5,0.214252,1,3,NaN,False,Franchise Dealer,NaN,nan
2,S Line,130839.34200,2.0,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,Audi A6,0.416667,2,5,416.0,False,Verified Trader,NaN,nan
3,SZ5,27654.00000,1.2,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,NaN,Suzuki Swift,-0.153409,1,1,174.0,False,Private Seller,NaN,nan
4,GT,19448.00000,1.5,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,NaN,1.84,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False,Franchise Dealer,4.5,4.5


In [92]:
#CONVERTING THE NUMBER STRINGS INTO ACTUAL NUMBERS

df2['Seller_Rating_Num'] = pd.to_numeric(df2['Seller_Rating_Clean'], errors='coerce')



df2.head(30)

,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Type,Seller_Rating,Seller_Rating_Clean,Seller_Rating_Num
0,Executive,205995.52000,1.7,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.00,NaN,Hyundai Tucson,0.160085,2,2,416.0,False,Private Seller,NaN,nan,NaN
1,NaN,72643.99826,2.0,2021.0,Automatic,NaN,Audi,Diesel,NaN,NaN,NaN,5,White,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.80,NaN,Audi Q5,0.214252,1,3,NaN,False,Franchise Dealer,NaN,nan,NaN
2,S Line,130839.34200,2.0,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.00,NaN,Audi A6,0.416667,2,5,416.0,False,Verified Trader,NaN,nan,NaN
3,SZ5,27654.00000,1.2,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.00,NaN,Suzuki Swift,-0.153409,1,1,174.0,False,Private Seller,NaN,nan,NaN
4,GT,19448.00000,1.5,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,NaN,1.84,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.00,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False,Franchise Dealer,4.5,4.5,4.5
5,Luna,80000.00000,1.0,2016.0,Manual,2,Toyota,Petrol,Hatchback,4,Medium Trim,5,Blue,180.0,4.31,3.95,1.51,1.70,Ireland,No,Franchise Dealer,"Macroom, Co. Cork",Yaris,13000.00,Co. Cork,Toyota Yaris,0.350000,2,4,416.0,False,Franchise Dealer,NaN,nan,NaN
6,R-Line,20800.00000,1.0,2022.0,Manual,2,Volkswagen,Petrol,Hatchback,5,Sports Trim,5,White,190.0,5.21,4.07,NaN,1.75,Ireland,No,Franchise Dealer\n4.6/5,Co. Galway,Polo,24945.00,Co. Galway,Volkswagen Polo,0.368182,2,1,447.0,False,Franchise Dealer,4.6,4.6,4.6
7,Premium,18461.00000,1.6,2023.0,Automatic,1,Hyundai,Petrol,SUV,5,Medium Trim,7,Grey,NaN,NaN,4.79,1.71,1.90,Ireland,No,Franchise Dealer,Co. Kerry,Santa Fe,57950.00,Co. Kerry,Hyundai Santa Fe,0.341522,2,4,843.0,False,Franchise Dealer,NaN,nan,NaN
8,Ambition,320000.00000,1.6,2016.0,Manual,3,Skoda,Diesel,Saloon,4,Medium Trim,5,Silver,180.0,4.11,4.66,1.46,1.81,Ireland,No,Private Seller\nNo rating yet,"North County, Co. Dublin",Octavia,6250.00,NaN,Skoda Octavia,0.116796,1,2,690.0,False,Private Seller,NaN,nan,NaN
9,Luna,105000.00000,1.6,2017.0,Manual,2,Toyota,Diesel,Saloon,4,Medium Trim,5,Grey,190.0,4.21,4.75,1.48,1.81,Ireland,No,Private Seller\nNo rating yet,"Ballina, Co. Mayo",Avensis,13100.00,NaN,Toyota Avensis,0.058966,1,2,690.0,False,Private Seller,NaN,nan,NaN


In [130]:
df2['Seller_Rating_Filled'] = df2['Seller_Rating_Num'].fillna(df2['Seller_Rating_Median'])

df2.head(30)

KeyError: 'Seller_Rating_Num'

In [93]:
#FILLING MISSING VALUES IN THE SELLER RATING COLUM WITH THE MEAN RATING

mean_rating = df2['Seller_Rating_Num'].mean()

df2['Seller_Rating_Mean'] = mean_rating

df2['Seller_Rating_Filled'] = df2['Seller_Rating_Num'].fillna(df2['Seller_Rating_Mean'])

df2.drop(['Seller_Rating_Mean','Seller_Rating_Median','Seller_Rating_Num','Seller_Rating_Clean'], axis=1, inplace=True)
df2.head(30)


KeyError: "['Seller_Rating_Median'] not found in axis"

In [94]:
df2.head()

,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Type,Seller_Rating,Seller_Rating_Clean,Seller_Rating_Num,Seller_Rating_Mean,Seller_Rating_Filled
0,Executive,205995.52000,1.7,2016.0,Manual,2,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,NaN,Hyundai Tucson,0.160085,2,2,416.0,False,Private Seller,NaN,nan,NaN,4.404229,4.404229
1,NaN,72643.99826,2.0,2021.0,Automatic,NaN,Audi,Diesel,NaN,NaN,NaN,5,White,NaN,NaN,NaN,NaN,NaN,UK,NaN,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,NaN,Audi Q5,0.214252,1,3,NaN,False,Franchise Dealer,NaN,nan,NaN,4.404229,4.404229
2,S Line,130839.34200,2.0,2018.0,Manual,1,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,NaN,Audi A6,0.416667,2,5,416.0,False,Verified Trader,NaN,nan,NaN,4.404229,4.404229
3,SZ5,27654.00000,1.2,2017.0,Automatic,3,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,NaN,Suzuki Swift,-0.153409,1,1,174.0,False,Private Seller,NaN,nan,NaN,4.404229,4.404229
4,GT,19448.00000,1.5,2024.0,Automatic,1,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,NaN,1.84,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False,Franchise Dealer,4.5,4.5,4.5,4.404229,4.500000


In [133]:
print(unique['Transmission'])


5


In [134]:
df2.to_excel('df-saved.xlsx', index=False)


In [44]:
df2.dtypes

Trim                       object
Mileage                   float64
Engine                     object
Year                      float64
Transmission               object
Total_Owners               object
Make                       object
Fuel_Type                  object
Body_Type                  object
Doors                      object
Trim_Level                 object
Seats                      object
Colour                     object
Road_Tax                  float64
Comb_Fuel_Consumption     float64
External_Length           float64
External_Height           float64
External_Width            float64
Current_Country_Of_Reg     object
Imported                   object
Independant_Dealer         object
Counties                   object
Model                      object
Price                      object
County                     object
combined                   object
sentiment                 float64
sentiment3                  int64
sentiment2                  int64
Days_Until_NCT

In [95]:
#REMOVING STRINGS FROM THE ENGINE COLUMN AND CONVERTING TO NUMERIC

df2['Engine'] = df2['Engine'].str.extract(r'(\d+\.?\d*)')

df2['Engine'] = pd.to_numeric(df2['Engine'], errors='coerce')
df2['Total_Owners'] = pd.to_numeric(df2['Total_Owners'], errors='coerce')





In [96]:


#GOING THROUGH ALL CATEGORICAL VARIABLES AND REPLACE NAN WITH "MISSING"

cat_cols = [
    'Trim', 'Transmission','Seats', 'Make', 'Fuel_Type', 'Body_Type',
    'Trim_Level', 'Colour', 'Current_Country_Of_Reg', 'Imported',
    'Counties','Doors', 'Model', 'County', 'combined', 'Seller_Type'
]

df2[cat_cols] = df2[cat_cols].fillna('Missing')



#GO THROUGH ALL NUMERICAL VARIABLES AND REPLACCE ALL NANS WITH MODE

columns_to_fill_nulls = [
    'Mileage',
    'Year',
    'Total_Owners',
    'Road_Tax',
    'Comb_Fuel_Consumption',
    'External_Length',
    'External_Height',
    'External_Width',
    'Days_Until_NCT',
    'Seller_Rating'
]

for col in columns_to_fill_nulls:
    median_val = df2[col].median()
    df2[col].fillna(median_val, inplace=True)

print(df2)

df2.head()

#COLUMNS THAT NEED TO BE CLEANED BEYOND THESE MEASURES



C:\Users\ttt56\AppData\Local\Temp\ipykernel_20672\1228536956.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2[col].fillna(median_val, inplace=True)
C:\Users\ttt56\AppData\Local\Temp\ipykernel_20672\1228536956.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Type,Seller_Rating,Seller_Rating_Clean,Seller_Rating_Num,Seller_Rating_Mean,Seller_Rating_Filled
0,Executive,205995.52000,1.7,2016.0,Manual,2.0,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,Missing,Hyundai Tucson,0.160085,2,2,416.0,False,Private Seller,4.6,nan,NaN,4.404229,4.404229
1,Missing,72643.99826,2.0,2021.0,Automatic,2.0,Audi,Diesel,Missing,Missing,Missing,5,White,200.0,4.81,4.41,1.49,1.81,UK,Missing,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,Missing,Audi Q5,0.214252,1,3,294.0,False,Franchise Dealer,4.6,nan,NaN,4.404229,4.404229
2,S Line,130839.34200,2.0,2018.0,Manual,1.0,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,Missing,Audi A6,0.416667,2,5,416.0,False,Verified Trader,4.6,nan,NaN,4.404229,4.404229
3,SZ5,27654.00000,1.2,2017.0,Automatic,3.0,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,Missing,Suzuki Swift,-0.153409,1,1,174.0,False,Private Seller,4.6,nan,NaN,4.404229,4.404229
4,GT,19448.00000,1.5,2024.0,Automatic,1.0,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,1.49,1.84,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False,Franchise Dealer,4.5,4.5,4.5,4.404229,4.500000


In [51]:
import re, nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
import joblib
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler






engine_median = df2['Engine'].median()

df2['Engine'] = df2['Engine'].fillna(engine_median)


#COUNT THE NULLS
null_columns = df2.isnull().sum()[df2.isnull().sum() > 0]
print(null_columns)













Independant_Dealer    19
dtype: int64


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ttt56\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ttt56\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ttt56\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ttt56\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [52]:
df2 = df2.drop('Seller_Rating_Num', axis=1)
df2 = df2.drop('Seller_Rating', axis=1)
df2 = df2.drop('Seller_Rating_Median', axis=1)
df2['Seller_Rating_Num'] = df2['Seller_Rating'].fillna(1)


df2.head(100)

KeyError: "['Seller_Rating_Num'] not found in axis"

In [97]:
#REPLACING SELLER RATING WITH 1 INSTEAD OF MEAN BECAUSE, SELLERS THAT DON'T HAVE RATINGS ARE MOST LIKELY INDEPENDANT DEALERS AND HAVE LOWER PRICES 

df2['Seller_Rating_Clean'] = df2['Seller_Rating_Clean'].replace('nan', np.nan).fillna(1)

df2.head(100)

,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Type,Seller_Rating,Seller_Rating_Clean,Seller_Rating_Num,Seller_Rating_Mean,Seller_Rating_Filled
0,Executive,205995.52000,1.7,2016.0,Manual,2.0,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.00,Missing,Hyundai Tucson,0.160085,2,2,416.0,False,Private Seller,4.6,1,NaN,4.404229,4.404229
1,Missing,72643.99826,2.0,2021.0,Automatic,2.0,Audi,Diesel,Missing,Missing,Missing,5,White,200.0,4.81,4.41,1.49,1.81,UK,Missing,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.80,Missing,Audi Q5,0.214252,1,3,294.0,False,Franchise Dealer,4.6,1,NaN,4.404229,4.404229
2,S Line,130839.34200,2.0,2018.0,Manual,1.0,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.00,Missing,Audi A6,0.416667,2,5,416.0,False,Verified Trader,4.6,1,NaN,4.404229,4.404229
3,SZ5,27654.00000,1.2,2017.0,Automatic,3.0,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.00,Missing,Suzuki Swift,-0.153409,1,1,174.0,False,Private Seller,4.6,1,NaN,4.404229,4.404229
4,GT,19448.00000,1.5,2024.0,Automatic,1.0,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,1.49,1.84,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.00,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False,Franchise Dealer,4.5,4.5,4.5,4.404229,4.500000
5,Luna,80000.00000,1.0,2016.0,Manual,2.0,Toyota,Petrol,Hatchback,4,Medium Trim,5,Blue,180.0,4.31,3.95,1.51,1.70,Ireland,No,Franchise Dealer,"Macroom, Co. Cork",Yaris,13000.00,Co. Cork,Toyota Yaris,0.350000,2,4,416.0,False,Franchise Dealer,4.6,1,NaN,4.404229,4.404229
6,R-Line,20800.00000,1.0,2022.0,Manual,2.0,Volkswagen,Petrol,Hatchback,5,Sports Trim,5,White,190.0,5.21,4.07,1.49,1.75,Ireland,No,Franchise Dealer\n4.6/5,Co. Galway,Polo,24945.00,Co. Galway,Volkswagen Polo,0.368182,2,1,447.0,False,Franchise Dealer,4.6,4.6,4.6,4.404229,4.600000
7,Premium,18461.00000,1.6,2023.0,Automatic,1.0,Hyundai,Petrol,SUV,5,Medium Trim,7,Grey,200.0,4.81,4.79,1.71,1.90,Ireland,No,Franchise Dealer,Co. Kerry,Santa Fe,57950.00,Co. Kerry,Hyundai Santa Fe,0.341522,2,4,843.0,False,Franchise Dealer,4.6,1,NaN,4.404229,4.404229
8,Ambition,320000.00000,1.6,2016.0,Manual,3.0,Skoda,Diesel,Saloon,4,Medium Trim,5,Silver,180.0,4.11,4.66,1.46,1.81,Ireland,No,Private Seller\nNo rating yet,"North County, Co. Dublin",Octavia,6250.00,Missing,Skoda Octavia,0.116796,1,2,690.0,False,Private Seller,4.6,1,NaN,4.404229,4.404229
9,Luna,105000.00000,1.6,2017.0,Manual,2.0,Toyota,Diesel,Saloon,4,Medium Trim,5,Grey,190.0,4.21,4.75,1.48,1.81,Ireland,No,Private Seller\nNo rating yet,"Ballina, Co. Mayo",Avensis,13100.00,Missing,Toyota Avensis,0.058966,1,2,690.0,False,Private Seller,4.6,1,NaN,4.404229,4.404229


In [99]:
df2.drop(columns=['Seller_Rating_Num', 'Seller_Rating_Mean', 'Seller_Rating_Filled','Seller_Rating'], inplace=True)
df2.head()

KeyError: "['Seller_Rating_Num', 'Seller_Rating_Mean', 'Seller_Rating_Filled'] not found in axis"

In [248]:
print(df2.shape)


(12537, 32)


In [101]:
df2 = df2.drop(columns=['Seller_Rating'])

df2.head()

,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Type,Seller_Rating_Clean
0,Executive,205995.52000,1.7,2016.0,Manual,2.0,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,10500.0,Missing,Hyundai Tucson,0.160085,2,2,416.0,False,Private Seller,1
1,Missing,72643.99826,2.0,2021.0,Automatic,2.0,Audi,Diesel,Missing,Missing,Missing,5,White,200.0,4.81,4.41,1.49,1.81,UK,Missing,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,32455.8,Missing,Audi Q5,0.214252,1,3,294.0,False,Franchise Dealer,1
2,S Line,130839.34200,2.0,2018.0,Manual,1.0,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,22500.0,Missing,Audi A6,0.416667,2,5,416.0,False,Verified Trader,1
3,SZ5,27654.00000,1.2,2017.0,Automatic,3.0,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,11000.0,Missing,Suzuki Swift,-0.153409,1,1,174.0,False,Private Seller,1
4,GT,19448.00000,1.5,2024.0,Automatic,1.0,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,1.49,1.84,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False,Franchise Dealer,4.5


In [102]:
from sklearn.model_selection import train_test_split

X = df2.drop(columns=['Price'])
Y = df2['Price']




In [103]:
X.head()
Y.head()

0    10500.0
1    32455.8
2    22500.0
3    11000.0
4    40995.0
Name: Price, dtype: float64

In [104]:
X.head()


,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Independant_Dealer,Counties,Model,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Type,Seller_Rating_Clean
0,Executive,205995.52000,1.7,2016.0,Manual,2.0,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,Private Seller\nNo rating yet,"Ardara, Co. Donegal",Tucson,Missing,Hyundai Tucson,0.160085,2,2,416.0,False,Private Seller,1
1,Missing,72643.99826,2.0,2021.0,Automatic,2.0,Audi,Diesel,Missing,Missing,Missing,5,White,200.0,4.81,4.41,1.49,1.81,UK,Missing,Franchise Dealer,"Dungannon, Co. Tyrone",Q5,Missing,Audi Q5,0.214252,1,3,294.0,False,Franchise Dealer,1
2,S Line,130839.34200,2.0,2018.0,Manual,1.0,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,Verified Trader\nNo rating yet,"Derry City, Co. Derry",A6,Missing,Audi A6,0.416667,2,5,416.0,False,Verified Trader,1
3,SZ5,27654.00000,1.2,2017.0,Automatic,3.0,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,Private Seller\nNo rating yet,"Greystones, Co. Wicklow",Swift,Missing,Suzuki Swift,-0.153409,1,1,174.0,False,Private Seller,1
4,GT,19448.00000,1.5,2024.0,Automatic,1.0,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,1.49,1.84,Ireland,No,Franchise Dealer\n4.5/5,Co. Dublin,3008,Co. Dublin,Peugeot 3008,0.333333,2,4,996.0,False,Franchise Dealer,4.5


In [105]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 103)


In [106]:
X.head()
Y.head()
X_train.head()
X_test.head() 
Y_train.head()
Y_test.head()

11925    22445.0
9011      2000.0
9235     63460.0
7637      1250.0
5353     17950.0
Name: Price, dtype: float64

In [107]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)


(10029, 32)
(2508, 32)
(10029,)
(2508,)


In [228]:
#DOWNLOADING X TRAIN AND Y TRAIN

df_X_train = pd.DataFrame(X_train)
df_X_test = pd.DataFrame(X_test)
df_Y_train = pd.DataFrame(Y_train)
df_Y_test = pd.DataFrame(Y_test)


df_X_train.to_excel('X_train.xlsx', index=False)
df_X_test.to_excel('X_test.xlsx', index=False)
df_Y_train.to_excel('Y_train.xlsx', index=False)
df_Y_test.to_excel('Y_test.xlsx', index=False)


print("All dataframes have been successfully exported to Excel files.")

All dataframes have been successfully exported to Excel files.


In [108]:
from pandas import get_dummies

#USING ONE HOT ENCODING TO ENCODE THE TRAINING SET AND TESTING SET SEPARATELY TO PREVENT DATA LEAKAGE, 
#THEN REALIGNING THE COLUMNS TO PREVENT A MISMATCH OF COLUMNS BETWEEN THE TRANING AND TEST SET


X_train["NCT_Expired"] = X_train["NCT_Expired"].map({True: 1, False: 0})


X_train_encoded = pd.get_dummies(X_train, columns=['Transmission','Doors','Seats','Body_Type','Fuel_Type',
                                                   'Trim_Level','Current_Country_Of_Reg','Imported',
                                                   'Seller_Type','Colour','Make'], dtype=int)

X_test_encoded = pd.get_dummies(X_test, columns=['Transmission','Doors','Seats','Body_Type','Fuel_Type',
                                                 'Trim_Level','Current_Country_Of_Reg','Imported',
                                                 'Seller_Type','Colour','Make'], dtype=int)


X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)


In [109]:
print(X_test_encoded.shape)
print(X_train_encoded.shape)
print(Y_train.shape)
print(Y_test.shape)




(2508, 152)
(10029, 152)
(10029,)
(2508,)


In [110]:
X_test_encoded.head()


,Trim,Mileage,Engine,Year,Total_Owners,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Independant_Dealer,Counties,Model,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Rating_Clean,Transmission_Automatic,Transmission_Manual,Transmission_Missing,Transmission_Tiptronic,Transmission_Unknown,Doors_0,Doors_1,Doors_2,Doors_3,Doors_4,Doors_5,Doors_6,Doors_8,Doors_Missing,Seats_1,Seats_2,Seats_3,Seats_4,Seats_5,Seats_6,Seats_7,Seats_8,Seats_9,Seats_Missing,Body_Type_Cabriolet,Body_Type_Chassis Cab,Body_Type_Convertible,Body_Type_Coupe,Body_Type_Crew Cab,Body_Type_Crossover,Body_Type_Estate,Body_Type_Estate/Jeep,Body_Type_Hatchback,Body_Type_Hearse,Body_Type_MPV,Body_Type_Minibus,Body_Type_Missing,Body_Type_Other,Body_Type_Pick Up,Body_Type_Pickup,Body_Type_SUV,Body_Type_Saloon,Body_Type_Unknown,Body_Type_Van,Body_Type_Wheelchair Accessible,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Hybrid,Fuel_Type_Missing,Fuel_Type_Petrol,Fuel_Type_Petrol Hybrid,Fuel_Type_Unknown,Trim_Level_Base Trim,Trim_Level_Luxury Trim,Trim_Level_Medium Trim,Trim_Level_Missing,Trim_Level_Sports Trim,Current_Country_Of_Reg_Ireland,Current_Country_Of_Reg_Missing,Current_Country_Of_Reg_Northern Ireland,Current_Country_Of_Reg_Other,Current_Country_Of_Reg_UK,Imported_Missing,Imported_N. Ireland,Imported_No,Imported_UK,Imported_Unknown,Imported_Unknown country,Seller_Type_Franchise Dealer,Seller_Type_Independent Dealer,Seller_Type_Missing,Seller_Type_No rating yet,Seller_Type_Private Seller,Seller_Type_Trader,Seller_Type_Verified Private Seller,Seller_Type_Verified Trader,Colour_Abyss Black,Colour_Ascari Blue Metallic,Colour_Aurora Grey,Colour_Beige,Colour_Black,Colour_Blue,Colour_Bronze,Colour_Brown,Colour_Burgundy,Colour_Cream,Colour_Gold,Colour_Green,Colour_Grey,Colour_Missing,Colour_Night Blue / Black Roof,Colour_Orange,Colour_Other,Colour_Phantom Black,Colour_Pink,Colour_Polar White,Colour_Purple,Colour_Red,Colour_Silky Bronze,Colour_Silver,Colour_Sunset Red,Colour_Two Tone,Colour_White,Colour_Yellow,Make_Alfa Romeo,Make_Audi,Make_BMW,Make_Citroen,Make_Dacia,Make_Fiat,Make_Ford,Make_Honda,Make_Hyundai,Make_Jaguar,Make_Kia,Make_Land Rover,Make_Lexus,Make_Mazda,Make_Mercedes-Benz,Make_Mini,Make_Mitsubishi,Make_Nissan,Make_Opel,Make_Peugeot,Make_Renault,Make_SEAT,Make_Skoda,Make_Suzuki,Make_Toyota,Make_Volkswagen,Make_Volvo
11925,Missing,42442.00000,1.0,2021.0,2.0,200.0,4.81,4.41,1.49,1.81,Franchise Dealer,"Sligo, Co. Sligo",Juke,Co. Sligo,Nissan Juke,0.090786,1,1,-99.0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9011,N,397506.98000,1.9,2004.0,2.0,673.0,4.81,4.41,1.49,1.81,Private Seller\nNo rating yet,Co. Sligo,A6,Missing,Audi A6,0.416667,2,5,21.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9235,Missing,102682.00000,2.0,2025.0,2.0,200.0,4.81,4.41,1.49,1.81,Franchise Dealer\n4.3/5,Co. Limerick,Tiguan,Co. Limerick,Volkswagen Tiguan,0.137240,1,2,294.0,0,4.3,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7637,SE,312821.00000,1.6,2011.0,1.0,280.0,5.31,4.53,1.63,1.84,Private Seller\nNo rating yet,"Castlebar, Co. Mayo",5008,Missing,Peugeot 5008,0.105548,2,3,-283.0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [111]:
X_train_encoded.head()

,Trim,Mileage,Engine,Year,Total_Owners,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Independant_Dealer,Counties,Model,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Rating_Clean,Transmission_Automatic,Transmission_Manual,Transmission_Missing,Transmission_Tiptronic,Transmission_Unknown,Doors_0,Doors_1,Doors_2,Doors_3,Doors_4,Doors_5,Doors_6,Doors_8,Doors_Missing,Seats_1,Seats_2,Seats_3,Seats_4,Seats_5,Seats_6,Seats_7,Seats_8,Seats_9,Seats_Missing,Body_Type_Cabriolet,Body_Type_Chassis Cab,Body_Type_Convertible,Body_Type_Coupe,Body_Type_Crew Cab,Body_Type_Crossover,Body_Type_Estate,Body_Type_Estate/Jeep,Body_Type_Hatchback,Body_Type_Hearse,Body_Type_MPV,Body_Type_Minibus,Body_Type_Missing,Body_Type_Other,Body_Type_Pick Up,Body_Type_Pickup,Body_Type_SUV,Body_Type_Saloon,Body_Type_Unknown,Body_Type_Van,Body_Type_Wheelchair Accessible,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Hybrid,Fuel_Type_Missing,Fuel_Type_Petrol,Fuel_Type_Petrol Hybrid,Fuel_Type_Unknown,Trim_Level_Base Trim,Trim_Level_Luxury Trim,Trim_Level_Medium Trim,Trim_Level_Missing,Trim_Level_Sports Trim,Current_Country_Of_Reg_Ireland,Current_Country_Of_Reg_Missing,Current_Country_Of_Reg_Northern Ireland,Current_Country_Of_Reg_Other,Current_Country_Of_Reg_UK,Imported_Missing,Imported_N. Ireland,Imported_No,Imported_UK,Imported_Unknown,Imported_Unknown country,Seller_Type_Franchise Dealer,Seller_Type_Independent Dealer,Seller_Type_Missing,Seller_Type_No rating yet,Seller_Type_Private Seller,Seller_Type_Trader,Seller_Type_Verified Private Seller,Seller_Type_Verified Trader,Colour_Abyss Black,Colour_Ascari Blue Metallic,Colour_Aurora Grey,Colour_Beige,Colour_Black,Colour_Blue,Colour_Bronze,Colour_Brown,Colour_Burgundy,Colour_Cream,Colour_Gold,Colour_Green,Colour_Grey,Colour_Missing,Colour_Night Blue / Black Roof,Colour_Orange,Colour_Other,Colour_Phantom Black,Colour_Pink,Colour_Polar White,Colour_Purple,Colour_Red,Colour_Silky Bronze,Colour_Silver,Colour_Sunset Red,Colour_Two Tone,Colour_White,Colour_Yellow,Make_Alfa Romeo,Make_Audi,Make_BMW,Make_Citroen,Make_Dacia,Make_Fiat,Make_Ford,Make_Honda,Make_Hyundai,Make_Jaguar,Make_Kia,Make_Land Rover,Make_Lexus,Make_Mazda,Make_Mercedes-Benz,Make_Mini,Make_Mitsubishi,Make_Nissan,Make_Opel,Make_Peugeot,Make_Renault,Make_SEAT,Make_Skoda,Make_Suzuki,Make_Toyota,Make_Volkswagen,Make_Volvo
4995,Comfortline,102682.00,1.6,2019.0,1.0,190.0,4.11,4.26,1.49,1.79,Private Seller\nNo rating yet,"Ballybofey, Co. Donegal",Golf,Missing,Volkswagen Golf,0.000000,1,5,478.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
12261,S Line,157000.00,2.0,2016.0,2.0,200.0,4.31,4.93,1.46,1.87,Independent Dealer,"Sligo, Co. Sligo",A6,Missing,Audi A6,0.416667,2,5,600.0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3230,Missing,104000.00,1.0,2014.0,2.0,200.0,4.81,4.41,1.49,1.81,Franchise Dealer,"Dungarvan, Co. Waterford",Polo,Co. Waterford,Volkswagen Polo,0.368182,2,1,294.0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7427,R-Line,87141.00,2.0,2022.0,2.0,190.0,4.61,4.28,1.49,1.79,Independent Dealer,"Portumna, Co. Galway",Golf,Co. Galway,Volkswagen Golf,0.000000,1,5,447.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [112]:
from category_encoders import TargetEncoder


#YOU DO TARGET ENCODING AFTER SEPARATING THE TARGET COLUMN

Target_Encoder = TargetEncoder(cols=['Trim', 'Counties', 'Model', 'combined', 'County'])
Target_Encoder.fit(X_train_encoded, Y_train)

X_train_encoded2 = Target_Encoder.transform(X_train_encoded)
X_test_encoded2 = Target_Encoder.transform(X_test_encoded)

X_train_encoded2.head()

,Trim,Mileage,Engine,Year,Total_Owners,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Independant_Dealer,Counties,Model,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Rating_Clean,Transmission_Automatic,Transmission_Manual,Transmission_Missing,Transmission_Tiptronic,Transmission_Unknown,Doors_0,Doors_1,Doors_2,Doors_3,Doors_4,Doors_5,Doors_6,Doors_8,Doors_Missing,Seats_1,Seats_2,Seats_3,Seats_4,Seats_5,Seats_6,Seats_7,Seats_8,Seats_9,Seats_Missing,Body_Type_Cabriolet,Body_Type_Chassis Cab,Body_Type_Convertible,Body_Type_Coupe,Body_Type_Crew Cab,Body_Type_Crossover,Body_Type_Estate,Body_Type_Estate/Jeep,Body_Type_Hatchback,Body_Type_Hearse,Body_Type_MPV,Body_Type_Minibus,Body_Type_Missing,Body_Type_Other,Body_Type_Pick Up,Body_Type_Pickup,Body_Type_SUV,Body_Type_Saloon,Body_Type_Unknown,Body_Type_Van,Body_Type_Wheelchair Accessible,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Hybrid,Fuel_Type_Missing,Fuel_Type_Petrol,Fuel_Type_Petrol Hybrid,Fuel_Type_Unknown,Trim_Level_Base Trim,Trim_Level_Luxury Trim,Trim_Level_Medium Trim,Trim_Level_Missing,Trim_Level_Sports Trim,Current_Country_Of_Reg_Ireland,Current_Country_Of_Reg_Missing,Current_Country_Of_Reg_Northern Ireland,Current_Country_Of_Reg_Other,Current_Country_Of_Reg_UK,Imported_Missing,Imported_N. Ireland,Imported_No,Imported_UK,Imported_Unknown,Imported_Unknown country,Seller_Type_Franchise Dealer,Seller_Type_Independent Dealer,Seller_Type_Missing,Seller_Type_No rating yet,Seller_Type_Private Seller,Seller_Type_Trader,Seller_Type_Verified Private Seller,Seller_Type_Verified Trader,Colour_Abyss Black,Colour_Ascari Blue Metallic,Colour_Aurora Grey,Colour_Beige,Colour_Black,Colour_Blue,Colour_Bronze,Colour_Brown,Colour_Burgundy,Colour_Cream,Colour_Gold,Colour_Green,Colour_Grey,Colour_Missing,Colour_Night Blue / Black Roof,Colour_Orange,Colour_Other,Colour_Phantom Black,Colour_Pink,Colour_Polar White,Colour_Purple,Colour_Red,Colour_Silky Bronze,Colour_Silver,Colour_Sunset Red,Colour_Two Tone,Colour_White,Colour_Yellow,Make_Alfa Romeo,Make_Audi,Make_BMW,Make_Citroen,Make_Dacia,Make_Fiat,Make_Ford,Make_Honda,Make_Hyundai,Make_Jaguar,Make_Kia,Make_Land Rover,Make_Lexus,Make_Mazda,Make_Mercedes-Benz,Make_Mini,Make_Mitsubishi,Make_Nissan,Make_Opel,Make_Peugeot,Make_Renault,Make_SEAT,Make_Skoda,Make_Suzuki,Make_Toyota,Make_Volkswagen,Make_Volvo
4995,16533.268632,102682.00,1.6,2019.0,1.0,190.0,4.11,4.26,1.49,1.79,Private Seller\nNo rating yet,16199.509784,16009.800514,12337.759955,16009.800514,0.000000,1,5,478.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
12261,37579.490834,157000.00,2.0,2016.0,2.0,200.0,4.31,4.93,1.46,1.87,Independent Dealer,34647.089241,27221.896569,12337.759955,27221.896569,0.416667,2,5,600.0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3230,26648.467955,104000.00,1.0,2014.0,2.0,200.0,4.81,4.41,1.49,1.81,Franchise Dealer,13316.838014,14518.590641,29230.725713,14518.590641,0.368182,2,1,294.0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7427,35115.979717,87141.00,2.0,2022.0,2.0,190.0,4.61,4.28,1.49,1.79,Independent Dealer,22586.634386,16009.800514,27769.714681,16009.800514,0.000000,1,5,447.0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,

In [113]:
print(X_test_encoded2.shape)
print(X_train_encoded2.shape)
print(Y_train.shape)
print(Y_test.shape)


(2508, 152)
(10029, 152)
(10029,)
(2508,)


In [242]:
print(X_train_encoded2.shape)
print(X_test_encoded2.shape)
print(X_train_encoded2.shape)
print(X_test_encoded2.shape)

(10029, 152)
(2508, 152)
(10029, 152)
(2508, 152)


In [235]:
X.head()

,Trim,Mileage,Engine,Year,Transmission,Total_Owners,Make,Fuel_Type,Body_Type,Doors,Trim_Level,Seats,Colour,Road_Tax,Comb_Fuel_Consumption,External_Length,External_Height,External_Width,Current_Country_Of_Reg,Imported,Counties,Model,Price,County,combined,sentiment,sentiment3,sentiment2,Days_Until_NCT,NCT_Expired,Seller_Type,Seller_Rating_Clean
0,Executive,205995.52000,1.7,2016.0,Manual,2.0,Hyundai,Diesel,SUV,5,Medium Trim,5,Black,200.0,4.41,4.48,1.65,1.85,Ireland,No,"Ardara, Co. Donegal",Tucson,10500.0,Missing,Hyundai Tucson,0.160085,2,2,419.0,False,Private Seller,1
1,Missing,72643.99826,2.0,2021.0,Automatic,2.0,Audi,Diesel,Missing,Missing,Missing,5,White,200.0,4.81,4.41,1.49,1.81,UK,Missing,"Dungannon, Co. Tyrone",Q5,32455.8,Missing,Audi Q5,0.214252,1,3,297.0,False,Franchise Dealer,1
2,S Line,130839.34200,2.0,2018.0,Manual,1.0,Audi,Diesel,Saloon,4,Sports Trim,5,Grey,200.0,4.41,4.93,1.46,1.87,Ireland,UK,"Derry City, Co. Derry",A6,22500.0,Missing,Audi A6,0.416667,2,5,419.0,False,Verified Trader,1
3,SZ5,27654.00000,1.2,2017.0,Automatic,3.0,Suzuki,Petrol,Hatchback,5,Luxury Trim,5,Black,200.0,4.51,3.84,1.50,1.74,Ireland,Unknown country,"Greystones, Co. Wicklow",Swift,11000.0,Missing,Suzuki Swift,-0.153409,1,1,177.0,False,Private Seller,1
4,GT,19448.00000,1.5,2024.0,Automatic,1.0,Peugeot,Diesel,MPV,4,Luxury Trim,5,Grey,270.0,5.31,4.45,1.49,1.84,Ireland,No,Co. Dublin,3008,40995.0,Co. Dublin,Peugeot 3008,0.333333,2,4,999.0,False,Franchise Dealer,4.5


In [114]:
#DOWNLOAD THE DATA

print(X_test_encoded2.shape)
print(X_train_encoded2.shape)
print(Y_train.shape)
print(Y_test.shape)


X_train_encoded2.to_excel("X_train_encoded_Fixed.xlsx", index=False)
X_test_encoded2.to_excel("X_test_encoded_Fixed.xlsx", index=False)
Y_train.to_excel("Y_train_Fixed.xlsx", index=False)
Y_test.to_excel("Y_test_Fixed.xlsx", index=False)


(2508, 152)
(10029, 152)
(10029,)
(2508,)
